In [ ]:
import torch
import torchaudio
import torchaudio.transforms as transforms 
import torch.nn as nn

import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import glob
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.signal import spectrogram
from scipy.fftpack import dst, dct

songs = '/home/george-vengrovski/Documents/canary_song_detector/sample_songs'
not_songs = '/home/george-vengrovski/Documents/canary_song_detector/sample_not_songs'

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Helper Functions

In [ ]:
import os
import shutil
import glob

def train_test_split(train_split=0.8):
    # Create directories if not already created
    if not os.path.exists('test'):
        os.makedirs('test')

    if not os.path.exists('train'):
        os.makedirs('train')

    # Get a list of all song files and non-song files
    songs = glob.glob('/home/george-vengrovski/Documents/canary_song_detector/sample_songs/*.wav')
    not_songs = glob.glob('/home/george-vengrovski/Documents/canary_song_detector/sample_not_songs/*.wav')

    # Split songs into train and test sets
    train_songs = songs[:int(len(songs) * train_split)]
    test_songs = songs[int(len(songs) * train_split):]

    # Copy song files into train and test directories with updated filenames
    for song in train_songs:
        file_name = os.path.basename(song)
        new_file_name = os.path.splitext(file_name)[0] + '_song.wav'
        shutil.copy(song, os.path.join('train', new_file_name))

    for song in test_songs:
        file_name = os.path.basename(song)
        new_file_name = os.path.splitext(file_name)[0] + '_song.wav'
        shutil.copy(song, os.path.join('test', new_file_name))

    # Split non-song files into train and test sets
    train_not_songs = not_songs[:int(len(not_songs) * train_split)]
    test_not_songs = not_songs[int(len(not_songs) * train_split):]

    # Copy non-song files into train and test directories with updated filenames
    for not_song in train_not_songs:
        file_name = os.path.basename(not_song)
        new_file_name = os.path.splitext(file_name)[0] + '_not_song.wav'
        shutil.copy(not_song, os.path.join('train', new_file_name))

    for not_song in test_not_songs:
        file_name = os.path.basename(not_song)
        new_file_name = os.path.splitext(file_name)[0] + '_not_song.wav'
        shutil.copy(not_song, os.path.join('test', new_file_name))
        
train_test_split()

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import matplotlib.pyplot as plt
import torchaudio.transforms as T
import random

class DataSet(Dataset):
    def __init__(self, type):
        song_files = []
        not_song_files = []

        folder = 'train' if type == 'train' else 'test' 

        for file in glob.glob(folder + '/*.wav'):
            if 'not_song' in file:
                not_song_files.append(file)
            else:
                song_files.append(file)

        # Combine all the files without sampling
        self.file_paths = song_files + not_song_files
        self.labels = [1]*len(song_files) + [0]*len(not_song_files)

        # Shuffle file_paths and labels together
        combined = list(zip(self.file_paths, self.labels))
        random.shuffle(combined)
        self.file_paths, self.labels = zip(*combined)

    def __getitem__(self, index):
        file_path = self.file_paths[index]
        label = self.labels[index]

        waveform, sample_rate = torchaudio.load(file_path)
        mean = waveform.mean()
        std = waveform.std()
        waveform = (waveform - mean) / std

        label = torch.tensor(label, dtype=torch.float32)  # labels are converted to a float tensor here

        return waveform, label

    def __len__(self):
        return len(self.file_paths)


    def collate_fn(self, batch):
        waveforms, labels = zip(*batch)

        # Pad the waveforms
        waveforms = pad_sequence([torch.flatten(w) for w in waveforms], batch_first=True)

        labels = torch.tensor(labels, dtype=torch.float32)  # labels should be a tensor, not a tuple
        
        return waveforms, labels

    
    def visualize_spectrogram(self, index):
        file_path = self.file_paths[index]
        waveform, sample_rate = torchaudio.load(file_path)

        # Create a spectrogram from the waveform
        spectrogram_transform = T.Spectrogram()
        spectrogram = spectrogram_transform(waveform)

        # Remove the channel dimension
        spectrogram = spectrogram.squeeze(0)

        # Plot the spectrogram
        plt.figure(figsize=(20, 10))
        plt.imshow(spectrogram.log2(), aspect='auto', origin='lower')
        plt.title('Spectrogram of ' + file_path)
        plt.xlabel('Frames')
        plt.ylabel('Frequency bins')
        plt.colorbar(format='%+2.0f dB')
        plt.tight_layout()
        plt.show()

    

train_dataset = DataSet(type='train')
train_dataset.visualize_spectrogram(0)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False, collate_fn=train_dataset.collate_fn)

test_dataset = DataSet(type='test')
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=test_dataset.collate_fn)

Classifier Architecture 
- 4 1d conv layers for the waveform of the periodicities, and amplitude 
- LSTM layers afterwards 
- Fully connected layer with two nodes for song v non-song 

In [ ]:
class Classifier(nn.Module):
    def __init__(self, feature_extractor_dim, hidden_size, num_layers):
        super(Classifier, self).__init__()

        # Feature extractor
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=feature_extractor_dim, kernel_size=10, padding=1, stride=5)
        self.conv2 = nn.Conv1d(in_channels=feature_extractor_dim, out_channels=feature_extractor_dim, kernel_size=4, padding=1, stride=4)
        self.conv3 = nn.Conv1d(in_channels=feature_extractor_dim, out_channels=feature_extractor_dim, kernel_size=4, padding=1, stride=4)
        self.conv4 = nn.Conv1d(in_channels=feature_extractor_dim, out_channels=feature_extractor_dim, kernel_size=4, padding=1, stride=4)
        self.conv5 = nn.Conv1d(in_channels=feature_extractor_dim, out_channels=feature_extractor_dim, kernel_size=4, padding=1, stride=4)
        self.conv6 = nn.Conv1d(in_channels=feature_extractor_dim, out_channels=feature_extractor_dim, kernel_size=3, padding=1, stride=3)
        self.conv7 = nn.Conv1d(in_channels=feature_extractor_dim, out_channels=feature_extractor_dim, kernel_size=3, padding=1, stride=3)

        # GRU Layer
        self.gru = nn.GRU(input_size=feature_extractor_dim, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

        # Fully connected layer 
        self.fc1 = nn.Linear(hidden_size*2, 1)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))

        # Reshape for GRU
        x = x.permute(0, 2, 1)

        # x is the output for each timestep of the GRU while h is the final hidden state
        x, h = self.gru(x)

        # squish the 2 directions into 1

        x = h.permute(1, 0, 2)
        x = x.flatten(start_dim=1)

        x = self.fc1(F.relu(x))
        # x = self.fc2(F.relu(x))
        # x = self.fc3(F.relu(x))
        x = torch.sigmoid(x)

        x = x.squeeze(0)
        
        return x 

In [ ]:
waveform, labels = next(iter(train_loader))

waveform = waveform.unsqueeze(1)

model = Classifier(feature_extractor_dim=64, hidden_size=200, num_layers=1)
print(model.forward(waveform).shape)


In [ ]:
from sklearn.metrics import balanced_accuracy_score
import matplotlib.pyplot as plt

epochs = 10
learning_rate = 0.0001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Classifier(feature_extractor_dim=16, hidden_size=64, num_layers=1).to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_preds = []
    train_labels = []
    for waveform, label in train_loader:
        waveform = waveform.unsqueeze(1).to(device)
        label = label.to(device)  # Changed here
        output = model(waveform)

        # Calculate accuracy
        predicted = (output > 0.5).float()
        train_preds.extend(predicted.detach().cpu().numpy())
        train_labels.extend(label.detach().cpu().numpy())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss = criterion(output, label)
        loss.backward()  # compute the gradients
        optimizer.step()

        train_loss += loss.item()

    train_losses.append(train_loss / len(train_loader))
    train_accuracy = balanced_accuracy_score(train_labels, train_preds) * 100
    train_accuracies.append(train_accuracy)
    print ('Epoch [{}/{}], Train Loss: {:.4f}, Train Accuracy: {:.2f}%'.format(epoch+1, epochs, train_loss / len(train_loader), train_accuracy))

    # Test the model
    model.eval()
    test_loss = 0.0
    test_preds = []
    test_labels = []
    with torch.no_grad():
        for waveform, label in test_loader:
            waveform = waveform.unsqueeze(1).to(device)
            label = torch.tensor(label, dtype=torch.float32).unsqueeze(1).to(device)
            output = model(waveform)

            # Calculate accuracy
            predicted = (output > 0.5).float()
            test_preds.extend(predicted.cpu().numpy())
            test_labels.extend(label.cpu().numpy())

            # Calculate loss
            loss = criterion(output, label)
            test_loss += loss.item()
        
    test_losses.append(test_loss / len(test_loader))
    test_accuracy = balanced_accuracy_score(test_labels, test_preds) * 100
    test_accuracies.append(test_accuracy)
    print('Epoch [{}/{}], Test Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch+1, epochs, test_loss / len(test_loader), test_accuracy))

# Plot the training and test losses over time
plt.figure(figsize=(10,5))
plt.title("Training and Test Loss")
plt.plot(train_losses,label="train")
plt.plot(test_losses,label="test")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Plot the training and test accuracies over time
plt.figure(figsize=(10,5))
plt.title("Training and Test Accuracy")
plt.plot(train_accuracies,label="train")
plt.plot(test_accuracies,label="test")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# save the weights
torch.save(model.state_dict(), 'model_weights.pth')

Feature Visualization

In [ ]:
import torch.optim as optim

iterations = 1000
learning_rate = .1

length = 44100 * 1 
channel = 0 

# Create a random tensor and make it require gradients
random_tensor = torch.randn(1, 1, length).to(device)
random_tensor.requires_grad = True
# Create an optimizer
optimizer = optim.Adam([random_tensor], lr=learning_rate)

model.train()
for i in range(iterations):
    # Zero the gradients
    optimizer.zero_grad()

    # Forward pass up to the first convolutional layer
    output = model(random_tensor)
    # output = F.relu(output[0].flatten(start_dim=1))

    # Select the activation of the first channel
    activation = output[0]

    # We want to maximize the activation, so we minimize the negative activation
    loss = -activation.mean()

    print(loss.item())

    # Backward pass
    loss.backward()

    # Update the input tensor
    optimizer.step()

print(random_tensor.shape)

In [1]:
import matplotlib.pyplot as plt
import librosa
import librosa.display

# Convert the tensor back to numpy and cpu for plotting
random_tensor_np = random_tensor.detach().cpu().numpy()

# As your tensor is likely a 3D tensor (batch_size, channels, length), 
# you might want to select one sample for visualization
selected_sample = random_tensor_np[0, 0, :]

# Compute the spectrogram
D = librosa.amplitude_to_db(np.abs(librosa.stft(selected_sample)), ref=np.max)

# Display the spectrogram
plt.figure(figsize=(10, 4))
librosa.display.specshow(D, sr=44100, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f Db')
plt.title('Spectrogram')
plt.show()


NameError: name 'random_tensor' is not defined